# 3.4章　記憶を持つエージェント

In [ ]:
# !pip install langchain
# !pip install langchain-openai

# !pip install serpapi
# !pip install google-search-results

# # load_toolsを利用するのに必要
# !pip install langchain_community

In [1]:
import os
# from google.colab import userdata
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['SERPAPI_API_KEY'] = os.getenv('SERPAPI_API_KEY')

# 3.4章

In [2]:
from langchain_core.prompts import PromptTemplate

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools']
template="""\
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Previous conversation history: {chat_history}
Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(input_variables=input_variables, template=template)
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Previous conversation history: {chat_history}
Question: {input}
Thought:{agent_scratchpad}


In [3]:
from langchain.memory import ChatMessageHistory

store = {}

def get_by_session_id(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [4]:
from langchain_openai import ChatOpenAI
from langchain.agents import load_tools
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.runnables.history import\
RunnableWithMessageHistory

model = ChatOpenAI(model="gpt-4o-mini")
tools = load_tools(["serpapi"], llm=model)
agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools,\
verbose=True)

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_by_session_id,
    input_messages_key="input",
    history_messages_key="chat_history",
)

response = agent_with_chat_history.invoke({"input": "株式会社\
Elithの住所を教えてください。最新の公式情報として公開されているものを教え\
てください。"},
    config={"configurable": {"session_id": "test-session1"}})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


最新の公式情報を取得するために、株式会社Elithの住所を検索します。  
Action: Search  
Action Input: 株式会社Elith 住所 公式サイト  ['株式会社エリスの公式HPです。 Elithは、最先端のAI技術をビジネス実装し価値を生み出す松尾研発スタートアップです.', '\u200bElith OFFICE · 東京都文京区本郷3-30-10本郷 K&Kビル1F · 東京メトロ丸ノ内線 / 本郷三丁目駅 徒歩約2分 都営大江戸線 / 本郷三丁目駅 徒歩約3分 東京メトロ千代田線 / ...', '株式会社Elith_オートクチュールAI開発_パンフレット202409. ソーシャルメディア. 連絡先. https://elith.co.jp contact@elith.co.jp 03-6822-5999. 住所. 日本113-0033 ...', '連絡先. https://www.elith.co.jp/ contact@elith.co.jp. 住所. 日本113-0033東京都文京区本郷3-30-10本郷 K&Kビル1F. この出展社を見ている人は、こちらの出展社も見てい ...', '井上 顧基 氏. 株式会社Elith Founder & CTO. 北陸先端科学技術大学院大学にて量子コンピュータの材料探索の研究で修士号を取得。会社経営と同時に東北 ...', '"AIの「今」をON AIR" 株式会社Elithが主催する、AIの最新動向についてラフに語るラジオです 「AIでこれから世の中はどう変わるのか？」「自分たちはAIをどのように ...', '- 会社名：株式会社Elith · - 代表者：若林怜帆人（代表取締役）、井上顧基（取締役CEO兼CTO） · - 所在地：東京都渋谷区恵比寿西2-3-13-601 · - 会社ウェブサイト：Elithの ...', '東大・松尾研発スタートアップの株式会社Elith(エリス)の公式アカウントです。 AI開発・研究を行っています。気軽にリプください、たくさん交流しましょう！', 'elith.co.jp/ contact@elith.co.jp. COMPANY ADDRESS. 日本113-0033東京都文京区本郷3-30-10本郷K&Kビル1F. STAND(S) SH

In [5]:
print(get_by_session_id("test-session1"))

Human: 株式会社Elithの住所を教えてください。最新の公式情報として公開されているものを教えてください。
AI: 東京都文京区本郷3-30-10本郷K&Kビル1F


In [6]:
response = agent_with_chat_history.invoke({"input": "先ほど尋ね\
た会社は何の会社ですか？"},
    config={"configurable": {"session_id": "test-session1"}})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


株式会社Elithについての情報を調べる必要があります。特にその業種や事業内容を知りたいです。  
Action: Search  
Action Input: 株式会社Elith 会社概要  [' entity_type: related_questions.', '事業内容：AI（人工知能）に関する研究、開発、設計、企画、教育、販売、保守、コンサルティング業務所在地：東京都渋谷区恵比寿西2-3-13-601\u200b', '株式会社エリスの公式HPです。 Elithは、最先端のAI技術をビジネス実装し価値を生み出す松尾研発スタートアップです.', '会社概要株式会社Elithは、東大松尾研発スタートアップ(R)として、最先端のAI技術を活用しビジネスに新たな価値を提供しています。国際学会発表・書籍出版の学術実績と ...', '株式会社Elithの魅力を伝えるコンテンツと、住所や代表・従業員などの会社情報です。ElithはAIのもっと身近な活用を推進して、より暮らしやすい世の中にします。', '東大・松尾研発スタートアップの株式会社Elith(エリス)の公式アカウントです。 AI開発・研究を行っています。気軽にリプください、たくさん交流しましょう！', '会社概要株式会社Elithは、東大松尾研発スタートアップ(R)として、最先端のAI技術を活用しビジネスに新たな価値を提供しています。国際学会発表・書籍出版の学術実績と ...', '製造・金融・医療業等向けに、生成AI利活用・LLM、画像AI開発・システム構築、AI教育アドバイザリー等のソリューションを提供。 決算書作成RPA対応AIを活用した自社株 ...', '株式会社Elithの概要. 住所, 〒 〒113-0033 東京都文京区本郷3-30-10本郷 K&Kビル1F. 分野, 映像認識・画像解析（視覚的）, 音声・音による入力（聴覚的）. 業界, 医療 ...', '株式会社Elith · 事業内容 · 株主. Not Available. 49 百万円. 評価額. 5 百万円. 累計調達額. 9999人. ログインして見る. 社員数 前月 ...', '□会社概要所在地：東京都文京区本郷3-30-10本郷 K&Kビル1F 代表者：代表取締役CEO 井上 顧基設立日：2022年12月会社HP：http

In [7]:
print(get_by_session_id("test-session1"))

Human: 株式会社Elithの住所を教えてください。最新の公式情報として公開されているものを教えてください。
AI: 東京都文京区本郷3-30-10本郷K&Kビル1F
Human: 先ほど尋ねた会社は何の会社ですか？
AI: 株式会社Elithは、AI技術の研究・開発を行い、ビジネスに新たな価値を提供するスタートアップです。


In [8]:
print(get_by_session_id("test-session2"))

In [9]:
response = agent_with_chat_history.invoke({"input": "先ほど尋ね\
た会社は何の会社ですか？"},
    config={"configurable": {"session_id": "test-session2"}})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


この質問では、先ほど尋ねた会社に関する情報を提供する必要がありますが、その会社についての具体的な情報がありません。どの会社について尋ねたのかを確認するために、関連する情報を探す必要があります。  
Action: Search  
Action Input: "先ほど尋ねた会社"  [' entity_type: related_questions.', '「社内の人間から聞いた」という状況を先方に説明するときの敬語表現を教えてください。例えば田中という部下が私にいるとして、 「田中から、あなた ...', '「お聞きしました」は、相手の言葉を尊重し、その情報を大切に受け取ったという事実を示すための表現です。ビジネスやフォーマルな場で特によく用いられ、 ...', '取引先や顧客の会社を訪問した際に「伺いました」という表現を使うと、訪問自体が相手に対して敬意を払っていることを表現できます。特に、フォーマル ...', '「伺っています」は、Xさんに敬意を表する言葉なので、不適切です。 逆に、XさんがBさん側の人である場合は、Xさんに敬意を表する表現が必要です。', '「聞く」という意味で伺うという言葉を使う場合、「担当者様より状況を伺っております」「先生のお話を伺って、大変勉強になりました」という表現法ができ ...', '「伺った」は「伺う」の過去形で、すでに「聞いた」「行った」「尋ねた」という行動が完了していることを表します。 ... たとえば、上司や取引先から情報を ...', 'お尋ねしました: 「尋ねる」は「質問する」や「調査する」などの意味も含むため、「お伺いしました」よりも幅広いシチュエーションで使用することができ ...', '何度もお聞きして申し訳ありません。 ＜個人情報流出を防ぐために、一度聞いたものは全て破棄(メールなら削除)しています。 これは結構重要です。', '「大変お世話になっております。株式会社〇〇の〇〇です。つきましては、先日弊社の〇〇から聞きました件でご連絡させて ...', 'こちらは、「聞く」「尋ねる」の敬語による表現について、取引先に対し、担当に直接聞いて（尋ねて）もらいたい旨伝える事例を題材に、楽しいイラストと会話形式で分かり ...']Action: Search  
Action Input: "先ほど尋ねた会社につい

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `I'm unable to determine the specific company you are inquiring about based on the provided information. If you could provide more context or details about the company, I would be better equipped to assist you.`

In [10]:
print(get_by_session_id("test-session2"))